# This notebook allows analyzing the friction coefficients (Additionally also `D` over time)

In [ ]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load plotting style
sys.path.append('/home/tobiac/polybox/studium/mscThesis/personal_repo/data_visualization/src')
from mscPlotting import set_plotting_environment, line_properties_generator, SizeParams, prepare_emittance_plots
set_plotting_environment()

# Load convergence plot tools
from mscPlotting import prepare_asymptotic_plots, order_lines, order_exp

# Lineplot Kwargs specific to this Notebook
lineplot_kwargs_dict = {
                   'logx' : True,
                   'logy' : True, 
                   'color' : None}

figure_dir = '/home/tobiac/polybox/studium/mscThesis/personal_repo/report/figures/tmp/'

## Plot the distribution of friction coefficient `Fd` for both DIH case and testcase

In [ ]:
Fd_dict = {"testPotentials" : pd.read_csv("FdNorm_it0064_testPotentials.csv"),
          "DIH" : pd.read_csv("FdNorm_it1199_DIH.csv")}
for key, df in Fd_dict.items():
    Fd_dict[key] = df.sort_values('v', inplace=False).drop_duplicates(subset='v')
    
    
    # Normalize x- and y-axis
    Fd_dict[key]['Fd/v'] /= Fd_dict[key]['Fd/v'].max()
    Fd_dict[key]['v'] /= Fd_dict[key]['v'].max()

In [ ]:
fig, ax = plt.subplots()

# Restart generator object for each plot and add current axis
lineplot_kwargs_dict['ax'] = ax
lineplot_kwargs_dict.pop('markersize', None)

lineplot_kwargs = line_properties_generator(lineplot_kwargs_dict)

# Load DIH sample
df = Fd_dict['DIH']
df.plot('v', 'Fd/v', kind='line', **next(lineplot_kwargs))

# Load Gaussian test case sample
df = Fd_dict['testPotentials']
df.plot('v', 'Fd/v', kind='line', **next(lineplot_kwargs))

first_ycol = 'Fd/v'
cols = df.columns
sizes = df[cols[0]].to_numpy()
y0 = df[first_ycol].iloc[0] + 35*df[first_ycol].iloc[0]
# y0 = df[first_ycol].to_numpy()[0]
intercepts = [y0]
slopes = [3.0]
order_lines(ax, sizes, intercepts, slopes)

x_label = r'$v$'
y_label = r'${\|\boldsymbol F_d\|}_2 / v$'

labels = ['DIH', 'Gaussian',r'$3^{\text{rd}}$ Order']
prepare_asymptotic_plots(ax, df['v'].to_numpy(), labels, x_label, y_label)

# ax.legend().remove()

# plt.savefig(figure_dir+f'Fd_asymptotic.pdf', bbox_inches="tight")

## Average Fd & D over time given `collision_statistics_0.csv` file

In [ ]:
collision_data = pd.read_csv("../../langevin_study/langevin_data/submission_data/langevin_P3M_Fd_D_r256_v64_vico_0712_1412/collision_statistics_0.csv").iloc[:1000]
FdAvg = collision_data['FdAvg'].to_numpy()

collision_data.insert(1, 'time', collision_data['iteration'].apply(lambda x: float(x)*2.15623e-13)) # [s]
collision_data.insert(2, 'tau_p', collision_data['time'].apply(lambda x: float(x)/4.3114e-11)) # [s]

collision_data.columns

In [ ]:
clip_time = 20

In [ ]:
fig, ax = plt.subplots()

lineplot_kwargs = line_properties_generator(lineplot_kwargs_dict)
size_params = SizeParams()

# Lineplot Kwargs
local_kwargs = lineplot_kwargs_dict.copy()
lineplot_kwargs_dict['ax'] = ax
lineplot_kwargs_dict['x'] = 'tau_p'
lineplot_kwargs_dict['logx'] = False
lineplot_kwargs_dict['logy'] = False

collision_data[:].plot(y='FdAvg', **next(lineplot_kwargs))

x_axis_label = r'Plasma Period $\tau_p$'
y_axis_label = r'$\langle F_d \rangle \  [cm/s^2]$'

prepare_emittance_plots(ax, x_axis_label, y_axis_label, legend_loc='top', bbox_to_anchor=(0.5, 1.45), plot_limit=False)

ax.get_legend().remove()

# plt.savefig(figure_dir+f'Fd_avg.pdf', bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots()

lineplot_kwargs = line_properties_generator(lineplot_kwargs_dict)
size_params = SizeParams()

# Lineplot Kwargs
local_kwargs = lineplot_kwargs_dict.copy()
lineplot_kwargs_dict['ax'] = ax
lineplot_kwargs_dict['x'] = 'tau_p'
lineplot_kwargs_dict['logx'] = False
lineplot_kwargs_dict['logy'] = True

# All coefficients
collision_data[clip_time:].plot(y='D0Avg_x', label=r'$\boldsymbol D_{xx}$', **next(lineplot_kwargs))
collision_data[clip_time:].plot(y='D0Avg_y', label=r'$\boldsymbol D_{xy}$', **next(lineplot_kwargs))
collision_data[clip_time:].plot(y='D0Avg_z', label=r'$\boldsymbol D_{xz}$', **next(lineplot_kwargs))
collision_data[clip_time:].plot(y='D1Avg_z', label=r'$\boldsymbol D_{yz}$', **next(lineplot_kwargs))

# Diagonal elements
# collision_data[clip_time:].plot(y='D0Avg_x', label=r'$\boldsymbol D_{xx}$', **next(lineplot_kwargs))
# collision_data[clip_time:].plot(y='D1Avg_y', label=r'$\boldsymbol D_{yy}$', **next(lineplot_kwargs))
# collision_data[clip_time:].plot(y='D2Avg_z', label=r'$\boldsymbol D_{zz}$', **next(lineplot_kwargs))

x_axis_label = r'Plasma Period $\tau_p$'
y_axis_label = r'$\langle |\boldsymbol D_{\circ\circ}| \rangle \  [cm^2/s^4]$'

prepare_emittance_plots(ax, x_axis_label, y_axis_label, legend_loc='top', bbox_to_anchor=(0.5, 1.22), plot_limit=False, ncol=4)

# plt.savefig(figure_dir+f'D_avg.pdf', bbox_inches="tight")